# Equation of state (EOS)

## Visualizing the WorkGraph Builder




## Prepare inputs and run


In [1]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.cp2k.eos import EosWorkgraph


load_profile()

atoms = bulk("Si")
structure = orm.StructureData(ase=atoms)
# code = orm.load_code("cp2k-ssmp-2024.1@localhost")
code = orm.load_code("cp2k-ssmp-2024@localhost")
# code = None
# Parameters.
parameters = {
    "GLOBAL": {
        "RUN_TYPE": "ENERGY_FORCE",
    },
    "FORCE_EVAL": {
        "METHOD": "Quickstep",
        "DFT": {
            "BASIS_SET_FILE_NAME": "BASIS_MOLOPT_UZH",
            "POTENTIAL_FILE_NAME": "POTENTIAL_UZH",
            "SCF": {
                "ADDED_MOS": 10,
                "SMEAR": {
                    "METHOD": "FERMI_DIRAC",
                    "ELECTRONIC_TEMPERATURE": 500,
                },
            },
            "KPOINTS": {
                "SCHEME": "MONKHORST-PACK 5 5 5",
            },
            "QS": {
                "EPS_DEFAULT": 1.0e-12,
                "METHOD": "GPW",
            },
            "MGRID": {
                "NGRIDS": 4,
                "CUTOFF": 500,
                "REL_CUTOFF": 50,
            },
            "XC": {
                "XC_FUNCTIONAL": {
                    "_": "PBE",
                },
            },
        },
        "SUBSYS": {
            "KIND": [
                {
                    "_": "Si",
                    "BASIS_SET": "DZVP-MOLOPT-PBE-GTH-q4",
                    "POTENTIAL": "GTH-PBE-q4",
                },
            ],
        },
    },
}
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
basis_pseudo_files = {
    "basis": orm.load_node(181682), # 181682, 9041
    "pseudo": orm.load_node(181683), # 181683, 9042
}
# ===============================================================================
wg = EosWorkgraph.build(
    structure=structure,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    scf_inputs = {
        "code": code,
        "parameters": orm.Dict(parameters),
        "file": basis_pseudo_files,
        "metadata": metadata,
    }
)
wg.to_html()


Run the workflow

In [2]:
wg.run()

09/04/2025 04:35:45 PM <1602706> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181806|WorkGraphEngine|continue_workgraph]: tasks ready to run: scale_structure
09/04/2025 04:35:46 PM <1602706> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181806|WorkGraphEngine|update_task_state]: Task: scale_structure, type: CALCFUNCTION, finished.
09/04/2025 04:35:46 PM <1602706> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181806|WorkGraphEngine|continue_workgraph]: tasks ready to run: all_scf
09/04/2025 04:35:49 PM <1602706> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181806|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 181815
09/04/2025 04:35:52 PM <1602706> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [181815|WorkGraphEngine|continue_workgraph]: tasks ready to run: Cp2kCalculation,Cp2kCalculation1,Cp2kCalculation2,Cp2kCalculation3,Cp2kCalculation4
09/04/2025 04:

Print out the results:

In [3]:
#------------------------- Print the output -------------------------
data = wg.outputs.result.value.get_dict()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))



Result: 
B: 88.464840164329
v0: 40.896274824518
e0: -214.09855023402
v0: 40.896274824518
